# Read me

Hello, This is for begginers who want to know how you can handle this dataset which consists of some csv file and image datasaet.

This notebook is just for handling data such as loading the dataset, stacking dataset, and etc...

It will be helpful for preparing analysis to someone who wants to submit.

Furthermore, I will upload the deep learning model through this data handling procedure for getting score.

If you have any questions, please leave the comments.


You can learn about using deep learning through my previous notebook (it is for titanic analysis).

## [Data handling & Deep learning]
 - https://www.kaggle.com/pythonash/how-to-handle-raw-dataset-and-analyze-with-dl
 
## [Preparing a completed dataset with proper imputation method]
 - https://www.kaggle.com/pythonash/making-completed-dataset
 
## [Deep learning model with SeLU activation function]
- https://www.kaggle.com/pythonash/selu-activation-function-in-dl

**Let's start!**

# Import some libraries for handling dataset.

In [ ]:
import pandas as pd
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
# seaborn is of use for visualizing.
import seaborn as sns

# load train, test, and submission sample dataset.
train_csv = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test_csv = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
submission = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')

## Glancing the train csv dataset.

All columns contain binary values except for 'Id' and 'Pawpularity'

In [ ]:
train_csv

Wow, there is no null-value.

So, we don't have to mind about imputation.

In [ ]:
train_csv.isnull().sum()

Furthermore, there is no duplicated data.

In [ ]:
train_csv.drop_duplicates()

## Count the binary values.

All data is unbalanced, but you don't have to worry about that.

I will expalin the reason why it is okay in next notebook (coming soon).

In [ ]:
for i in train_csv.drop(['Id','Pawpularity'],axis=1):
    sns.countplot(train_csv[i])
    plt.show()

The target variable of train dataset is distributed as below.

In this part, you have to determine whether you truncate some outliers or not.

However, I recommend that you don't truncate because I think those are a part of dataset, too.

We don't know what the truncated data affects in using deep learning.

It's just on my experience so, you can select and it's all up to you!

In [ ]:
sns.distplot(train_csv['Pawpularity'])

## Glancing the test dataset.

In [ ]:
test_csv

It has not null-values, either.

In [ ]:
test_csv.isnull().sum()

## Finally, Let's identify the submission form.

In [ ]:
submission

# Handling image dataset.

In [ ]:
# Set the path for loading image dataset.
os.chdir('../input/petfinder-pawpularity-score/train')

# We can find the size of each image data from this procedure
size_data = pd.DataFrame()
for file in os.listdir():
    imgg = cv2.imread(file)
    w,h,c = imgg.shape
    size_data=size_data.append([[w,h,c,imgg.size/3]])
size_data

## What is the minimum size?

In [ ]:
size_data[size_data[3] == size_data[3].min()]

## What is the number of most things?

In [ ]:
size_data[3].value_counts()

## Checking the pixel structure of that.

- It is derived from 960 * 720.

In [ ]:
size_data[size_data[3] == 691200]

## Load train image dataset and rescaling.

This part is crucial for analysis and is devided into 3 parts.

1. Loading the image dataset.

2. Changing the shape of each image into 64 * 64.

- Why 64 * 64 ?? the reason is that you will stuck in memory allocation.

- That is, there are so many image dataset which make your memory be exploded.

- So, reszing the image into 64 * 64, you can take your memory with sufficient.

3. Rescaling the pixels by deviding with 255.

In this procedure, the parameter, cv2.INTER_AREA, is useful for interpolation.

There are several interpolation methods, but I recommend this.

In [ ]:
train_img = []
for i in os.listdir():
    file = cv2.imread(i)
    file=cv2.resize(file,(64,64), interpolation=cv2.INTER_AREA)
    train_img.append(file/255)
train_img[:5]

## For corresponding the image with each id.

Each image data has to correspond to its own id.

In [ ]:
train_img_name = []
for i in os.listdir():
    train_img_name.append(i)
train_img_name[:5]

## Checking the file name.

Before we match the image data with its own id, we have to check the file name whether file name has identical rule or not.

If a file name has not '.jpg', it will be shown.

In [ ]:
for name in train_img_name:
    if name[-4:] != '.jpg':
        print(name)

## Matching the image dataset order with csv file order.

In [ ]:
train_csv_data = pd.DataFrame()
for img, name in zip(train_img, train_img_name):
    name=name[:-4]
    location = train_csv[train_csv['Id'] == name].index[0]
    train_csv_data= train_csv_data.append([train_csv.loc[location]])
train_csv_data

Reindexing the train csv data.

In [ ]:
train_csv_data=train_csv_data.reset_index().drop(['index'],axis=1)
train_csv_data

## Checking the resized and rescaled image with original image data.

It's original data.

In [ ]:
image_1 = cv2.imread('./'+train_csv_data['Id'][0]+'.jpg')
plt.imshow(image_1)

It's resized and rescaled data.

In [ ]:
plt.imshow(train_img[0])

It's original data.

In [ ]:
image_2 = cv2.imread('./'+train_csv_data['Id'][1]+'.jpg')
plt.imshow(image_2)

It's resized and rescaled data.

In [ ]:
plt.imshow(train_img[1])

## Load test image dataset and rescaling

What is the shape of test dataset??

It's (128, 128, 3), but we have to resize the test image into 64 * 64 because of memory allocation problem.

For analyzing the test dataset, we will match the sizes of both train image and test image, identically.

In [ ]:
os.chdir('../test')

for i in os.listdir():
    file = cv2.imread(i)
    print(file.shape)

In [ ]:
test_img = []
for i in os.listdir():
    file = cv2.imread(i)
    file=cv2.resize(file,(64,64), interpolation=cv2.INTER_AREA)
    test_img.append(file/255)
test_img[:5]

Below procedure is identical with train dataset.

In [ ]:
test_img_name = []
for i in os.listdir():
    test_img_name.append(i)
test_img_name[:5]

In [ ]:
test_csv_data = pd.DataFrame()
for img, name in zip(test_img, test_img_name):
    name=name[:-4]
    location = test_csv[test_csv['Id'] == name].index[0]
    test_csv_data= test_csv_data.append([test_csv.loc[location]])
test_csv_data=test_csv_data.reset_index().drop(['index'],axis=1)
test_csv_data

In [ ]:
test_1 = cv2.imread('./'+test_csv_data['Id'][0]+'.jpg')
plt.imshow(test_1)

In [ ]:
plt.imshow(test_img[0])

## Prepare the train input, target and test input.

In [ ]:
train_csv_x = train_csv_data.drop(['Id','Pawpularity'],axis=1)
train_y = train_csv_data['Pawpularity']

test_csv_x = test_csv_data.drop(['Id'],axis=1)

# It has done! 

You are ready to use this dataset as train and test for getting score.

I think you may want to know why test image doesn't seem like animal.

And, you will wonder whether this dataset can be a useful resource for analyzing or not.

The clue will be on next notebook.

Anyway, fingers crossed!